# Interpolate Sticky-Delta Volatility with Python

In OTC FX option market, volatility smiles are quoted in a delta-volatilty instead of strike-volatility convention in equity option market. For pricing a given option, when one prepares volatility as a parameter in a Black-Scholes formula, they need to calculate the volatility for the given strike price in a delta-volatility dimension matrix.

## 1. Quoted in Delta (premium excluded)
Premium excluded delta is the case where the premium is paid in the quoted currency (ccy2) of a currency pair. Here the deltas can be used for interpolation without any adjustment.

First, setup the market data.
You may notice that negative deltas are used, which means they are delta of put. 

In [1]:
import QuantLib as ql
import math

as_of_date = ql.Date(1,2,2022)
# Market Data
tenors = [ql.Period('1M'), ql.Period('3M'), ql.Period('6M')]
maturities = [as_of_date + t for t in tenors]
voltilities = [[0.040, 0.030, 0.020, 0.025, 0.035], 
               [0.045, 0.040, 0.030, 0.035, 0.042],
               [0.060, 0.052, 0.035, 0.045, 0.048],
              ]
deltas = [[-0.90, -0.75, -0.5, -0.25, -0.10], 
          [-0.90, -0.75, -0.5, -0.25, -0.10], 
          [-0.90, -0.75, -0.5, -0.25, -0.10], 
         ]
rTS = ql.FlatForward(ql.Date(1,2,2022), ql.QuoteHandle(ql.SimpleQuote(0.05)), ql.Actual360(), ql.Compounded, ql.Annual)
qTS = ql.FlatForward(ql.Date(1,2,2022), ql.QuoteHandle(ql.SimpleQuote(0.01)), ql.Actual360(), ql.Compounded, ql.Annual)
spot = 1.1
strike = 1.2

Setup a target function for recurrsion

In [2]:
class TargetFun:
    def __init__(self, as_of_date, spot, rdf, qdf, strike, maturity, deltas, smile):
        self.ref_date = as_of_date
        self.strike = strike
        self.maturity = maturity
        self.spot = spot
        self.rDcf = rdf # discount factor
        self.qDcf = qdf # discount factor
        self.t = ql.Actual365Fixed().yearFraction(as_of_date, self.maturity)
        self.interp = ql.LinearInterpolation(deltas, smile)
        self.delta_type = ql.DeltaVolQuote.Spot
        
    def __call__(self, v0):
        optionType = ql.Option.Put
        stdDev = math.sqrt(self.t) * v0
        calc = ql.BlackDeltaCalculator(optionType, self.delta_type, self.spot, self.rDcf, self.qDcf, stdDev)
        d = calc.deltaFromStrike(self.strike)
        v = self.interp(d, allowExtrapolation=True)
        return (v - v0)

Set up a solver

In [3]:
solver = ql.Brent()
accuracy = 1e-12
step = 1e-6

In [4]:
vol_pts = []
for i, smile in enumerate(voltilities):
    mat = maturities[i]
    target = TargetFun(as_of_date, 
                       spot,
                       rTS.discount(mat), 
                       qTS.discount(mat), 
                       strike,
                       maturities[i], 
                       deltas[i],
                       smile)
    guess = smile[2]
    vol_pts.append(solver.solve(target, accuracy, guess, step))

vts = ql.BlackVarianceCurve(as_of_date, 
                            maturities,
                            vol_pts, 
                            ql.Actual365Fixed(), 
                            False)
vts.enableExtrapolation()
vts_handle = ql.BlackVolTermStructureHandle(vts)